# PyTorch test for Overlpap (Skl) Integrals and Gradients 

In [1]:

import torch as th

import time

In [2]:
#dtype = th.float64   # Use float32 if you are on GeForce GPU
dtype = th.cfloat
gpuid = 0
#device = th.device("cuda:"+ str(gpuid))
device = th.device("cpu")  # un-comment to change back to CPU

print("Execution device: ",device)
print("PyTorch version: ", th.__version__ )
print("CUDA available: ", th.cuda.is_available())
print("CUDA version: ", th.version.cuda)
print("CUDA device:", th.cuda.get_device_name(gpuid))

Execution device:  cpu
PyTorch version:  1.8.0.dev20201112
CUDA available:  True
CUDA version:  11.0
CUDA device: TITAN V


In [107]:
# Utility functions

# return the lower triangle of A in column order i.e. vech(A)
def vech(A):
    count = 0
    c = A.shape[0]
    v = th.zeros(c * (c + 1) // 2,  device=device, dtype=dtype)
    for j in range(c):
        for i in range(j,c):
            v[count] = A[i,j]
            count += 1
    #return th.tensor(v , device=device, dtype=dtype)
    return v

# vech2L   create lower triangular matrix L from vechA
def vech2L(v,n):
    count = 0
    L = th.zeros((n,n), device=device, dtype=dtype)
    for j in range(n):
        for i in range(j,n):
            L[i,j]=v[count]
            count += 1
    #return th.tensor(L , device=device, dtype=dtype)
    return L

# need to define prod by hand since it is not grad safe in pytorch
def myprod(v):
    p = 1
    for i in v:
        p *= i
    return p

In [111]:
def matrix_elements(n, vechLk, vechLl, Sym):
    

    Lk = vech2L(vechLk,n);
    Ll = vech2L(vechLl,n);
    
    # apply symmetry projection on Ll
    
    # th.t() is shorthand for th.transpose(X, 0,1)
    PLl = th.t(Sym) @ Ll;
    
    # build Ak, Al, Akl, invAkl, invAk, invAl

    Ak = Lk@th.conj(th.t(Lk));
    Al = PLl@th.conj(th.t(PLl));
    Akl = Ak+Al;
    
    invAkl = th.inverse(Akl);
    detLk = myprod(th.diagonal(Lk))
    detLl = myprod(th.diagonal(Ll))
    detAkl = myprod(th.diagonal(th.cholesky(Akl)))**2
    
    # Overlap: (normalized)
    #skl = 2**(3*n/2) * th.sqrt( th.pow(th.abs(th.det(Lk))*th.abs(th.det(Ll))/th.det(Akl) ,3) );
    skl = 2**(3*n/2) * th.sqrt( th.pow(th.abs(detLk)*th.abs(detLl)/detAkl ,3) );

    #Analytic gradient formulas with respect to vechLk vechLl
    checkdsk = vech( 3/2 * skl * (th.diag(1/th.diag(Lk)) - 2*invAkl@Lk) )
    checkdsl = vech( 3/2 * skl * (th.diag(1/th.diag(Ll)) - 2*Sym@invAkl@PLl) )
    
    # Now get the gradient using autograd
    dsk = 0 #th.autograd.grad(skl, vechLk, retain_graph=True)
    # 
    dsl = 0 #th.autograd.grad(skl, vechLl)

    
    return {'skl':skl, 'dsk':dsk, 'dsl':dsl, 'checkdsk':checkdsk, 'checkdsl':checkdsl}

In [112]:
def test_matrix_elements():
    n = 3;
    
    # using numbers that I know the correct results for
    vechLk = th.tensor([  1.00000039208682 + 1.0j, 
              0.02548044275764261+ 1.0j, 
              0.3525161612610669+ 1.0j,
              1.6669144815242515+ 1.0j,
              0.9630555318946559+ 1.0j,
              1.8382882034659822+ 1.0j ], device=device, dtype=dtype, requires_grad=True);
    
    vechLl = th.tensor([  1.3353550436464964+ 1.0j,
               0.9153272033682132+ 1.0j,
               0.7958636766525028+ 1.0j,
               1.8326931436447955+ 1.0j,
               0.3450426931160630+ 1.0j,
               1.8711839323167831+ 1.0j ], device=device, dtype=dtype, requires_grad=True);
    
    Sym = th.tensor([[0,0,1],
                    [0,1,0],
                    [1,0,0]], device=device, dtype=dtype);
 
    matels = matrix_elements(n, vechLk, vechLl, Sym) 
    
    print('skl:      ',matels['skl'])
    print('dsk:      ',matels['dsk'])
    print('checkdsk:  ',matels['checkdsk'])
    print('dsl:      ',matels['dsl'])
    print('checkdsl:  ',matels['checkdsl'])
    

In [113]:
#start_time = time.time()
#for i in range(1000):
#    test_matel()
#print(" took {} seconds ".format(time.time() - start_time))

test_matrix_elements()

RuntimeError: sqrt does not support automatic differentiation for outputs with complex dtype.

In [57]:
test_matrix_elements()

RuntimeError: cholesky_cpu: U(2,2) is zero, singular U.

In [45]:
vechLk = th.tensor([  1.00000039208682 + 1.0j, 
              0.02548044275764261+ 1.0j, 
              0.3525161612610669+ 1.0j,
              1.6669144815242515+ 1.0j,
              0.9630555318946559+ 1.0j,
              1.8382882034659822+ 1.0j ], device=device, dtype=dtype, requires_grad=False);

In [46]:
vechLl = th.tensor([  1.3353550436464964+ 1.0j,
               0.9153272033682132+ 1.0j,
               0.7958636766525028+ 1.0j,
               1.8326931436447955+ 1.0j,
               0.3450426931160630+ 1.0j,
               1.8711839323167831+ 1.0j ], device=device, dtype=dtype, requires_grad=False);

In [47]:
Sym = th.tensor([[0,0,1],
                    [0,1,0],
                    [1,0,0]], device=device, dtype=dtype);

In [48]:
n=3
Lk = vech2L(vechLk,n);
Ll = vech2L(vechLl,n);
vechLl

tensor([1.3354+1.j, 0.9153+1.j, 0.7959+1.j, 1.8327+1.j, 0.3450+1.j, 1.8712+1.j])

In [49]:
#PLl = th.t(Sym) @ Ll;
th.t(Sym) @ Ll

tensor([[0.7959+1.j, 0.3450+1.j, 1.8712+1.j],
        [0.9153+1.j, 1.8327+1.j, 0.0000+0.j],
        [1.3354+1.j, 0.0000+0.j, 0.0000+0.j]])

In [95]:
W = th.rand(3,3)
#+th.eye(3)
W = th.mm(W, th.t(W))

In [96]:
th.det(W)

tensor(0.2346)

In [97]:
#Wd=th.diag(th.cholesky(W))
th.prod(th.diag(th.cholesky(W)))**2

tensor(0.2346)

In [98]:
th.inverse(W)

tensor([[ 3.4700, -2.2685,  0.0946],
        [-2.2685,  2.5788, -1.4170],
        [ 0.0946, -1.4170,  2.7998]])